In [5]:
# import math
# import torchtext
# import torch
# import torch.nn as nn
# from torchtext.data.utils import get_tokenizer
# from collections import Counter
# from torchtext.vocab import Vocab
# from torchtext.utils import download_from_url,extract_archive
# from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import DataLoader
# from torch import Tensor
# from torch.nn import (TransformerEncoder,TransformerDecoder,TransformerEncoderLayer,TransformerDecoderLayer)
# import io
# import time
# #https://www.analyticsvidhya.com/blog/2021/06/language-translation-with-transformer-in-python/

In [6]:
# import io
# import torch
# from collections import Counter
# from torchtext.vocab import Vocab
# from torchtext.utils import download_from_url, extract_archive
# from spacy.lang.de import German
# from spacy.lang.en import English

# url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
# train_urls = ('train.de.gz', 'train.en.gz')
# val_urls = ('val.de.gz', 'val.en.gz')
# test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')
# train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
# val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
# test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

# de_tokenizer = German().tokenizer
# en_tokenizer = English().tokenizer

# def build_vocab(filepath, tokenizer):
#     counter = Counter()
#     with io.open(filepath, encoding='utf8') as f:
#         for string_ in f:
#             counter.update(tokenizer(string_))
#     return Vocab(counter)

# de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
# en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

# def data_process(filepaths, de_vocab, en_vocab, de_tokenizer, en_tokenizer):
#     print('Processing files:', filepaths)
#     raw_de_iter = iter(io.open(filepaths[0], encoding='utf8'))
#     raw_en_iter = iter(io.open(filepaths[1], encoding='utf8'))
#     data = []
#     for raw_de, raw_en in zip(raw_de_iter, raw_en_iter):
#         de_tensor = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de.rstrip('\n'))], dtype=torch.long)
#         en_tensor = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en.rstrip('\n'))], dtype=torch.long)
#         if len(de_tensor) == 0 or len(en_tensor) == 0:
#             print('Warning: Empty tensor found!')
#             continue
#         data.append((de_tensor, en_tensor))
#     print('Number of samples:', len(data))
#     print(data)
#     return data

# train_data = data_process(train_filepaths, de_vocab, en_vocab, de_tokenizer, en_tokenizer)
# val_data = data_process(val_filepaths, de_vocab, en_vocab, de_tokenizer, en_tokenizer)
# test_data = data_process(test_filepaths, de_vocab, en_vocab, de_tokenizer, en_tokenizer)


In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
#This code appears to be a method of a Python class that is responsible for building a vocabulary of words. It takes a word as input and updates several attributes of the class to incorporate the new word into the vocabulary.

#The addWord method first checks if the word is already present in the word2index dictionary. If the word is not in the dictionary, it means that the word is new and needs to be added to the vocabulary. In this case, the method adds the word to the word2index dictionary and assigns it a new index that corresponds to the current size of the vocabulary. The method then initializes the word2count dictionary for this word, setting the count to 1, since this is the first time this word has been seen. Finally, the method adds the word to the index2word dictionary, mapping the index to the word, and increments the total number of words in the vocabulary, n_words, by 1.

#On the other hand, if the word is already in the word2index dictionary, it means that the word has already been added to the vocabulary before. In this case, the method simply increments the count of the word in the word2count dictionary.

#Overall, this method is used as part of a larger process for building a vocabulary of words, which is often an important preprocessing step in natural language processing tasks such as text classification, machine translation, and text generation.

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    ) #Removing diactirc symbols (´), grave accents (`), circumflex accents (^),

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

#This is a function called readLangs that reads and processes data from a file containing parallel text in two languages, lang1 and lang2. The function returns two Lang instances representing the input and output languages, respectively, as well as a list of sentence pairs, where each pair consists of a sentence in lang1 and its corresponding translation in lang2.

#The function begins by opening the file located in the data directory with the name lang1-lang2.txt, where lang1 and lang2 are the input parameters. The file is assumed to contain one sentence pair per line, separated by a tab character.

#The read() method is called to read the contents of the file as a single string, and the strip() method is called to remove any leading or trailing whitespace characters. The resulting string is then split into a list of lines using the split('\n') method.

#The pairs variable is initialized as a list comprehension that iterates over the lines in the file and splits each line into a pair of sentences using the split('\t') method. The normalizeString function is called on each sentence in the pair to remove any unwanted characters or diacritics, and the resulting normalized sentences are stored as a new pair in the pairs list.

#If the reverse parameter is set to True, the order of the sentence pairs is reversed, so that the translation in lang2 comes first and the original sentence in lang1 comes second. In this case, the pairs list is modified by reversing the order of the sentences in each pair using the reversed() function.

#Two Lang instances are then created to represent the input and output languages, respectively, depending on the value of the reverse parameter. If reverse is True, the Lang instance for the input language is created with the name lang2, and the Lang instance for the output language is created with the name lang1. Otherwise, the input and output languages are created with the names lang1 and lang2, respectively.

#Finally, the function returns the two Lang instances and the list of sentence pairs.

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


#This is a Python code that defines two functions: filterPair and filterPairs.

#The filterPair function takes a pair of sentences (as a tuple) as input and returns True if the length of the first sentence (the source sentence) and the second sentence (the target sentence) are both less than MAX_LENGTH, which is set to 10 in this code, and if the second sentence starts with any of the English prefixes defined in eng_prefixes.

#The filterPairs function takes a list of pairs of sentences as input and returns a new list of pairs that meet the conditions defined in filterPair. It uses a list comprehension to iterate over each pair in the input list and filter out any pairs that do not meet the conditions.

#Overall, these functions are used to filter out pairs of sentences that are too long or do not match the expected format, based on the English prefixes defined in the eng_prefixes tuple. These filtered pairs can be used for various natural language processing tasks, such as machine translation or language modeling.

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['nous sommes en train de lire .', 'we re reading .']


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    
#This is a PyTorch implementation of an Encoder RNN (Recurrent Neural Network) class. The Encoder RNN is a type of neural network used in natural language processing and machine translation tasks to encode a sequence of words into a fixed-length vector representation.

#The __init__ method initializes the Encoder RNN class and defines its components. The input_size parameter is the size of the input vocabulary (i.e., the number of unique words in the input language), and hidden_size is the size of the hidden state of the RNN.

#The embedding layer is used to map the input words to a continuous vector space of size hidden_size. This is necessary because the RNN cannot work with discrete inputs. The gru layer is a gated recurrent unit that processes the input sequence and updates its hidden state at each time step.

#The forward method defines the forward pass of the Encoder RNN. It takes as input an input tensor representing a sequence of words and a hidden tensor representing the initial hidden state of the RNN. The input is first passed through the embedding layer to obtain a continuous vector representation. The resulting vector is then passed through the gru layer, which updates the hidden state of the RNN and produces an output tensor. The output tensor and the updated hidden state are returned.

#The initHidden method initializes the hidden state of the RNN with zeros. It returns a tensor of shape (1, 1, hidden_size) representing the initial hidden state.

#n a Recurrent Neural Network (RNN), the hidden state is the network's internal state that contains information about the previous inputs processed by the network. At each time step, the RNN takes an input and updates its hidden state based on the current input and the previous hidden state. 
#The hidden state can be seen as a compressed representation of the past inputs that the network has processed.

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    
#This code defines a class called DecoderRNN that inherits from nn.Module. This class is used for implementing a recurrent neural network (RNN) for decoding a sequence of output tokens.

#The __init__ method initializes the parameters for the decoder RNN. The hidden_size is the number of features in the hidden state of the RNN, and output_size is the number of possible output tokens.

#The decoder RNN has four layers:

#nn.Embedding(output_size, hidden_size) initializes an embedding layer with output_size inputs and hidden_size outputs. It is used to convert the input tokens into a continuous vector space.

#nn.GRU(hidden_size, hidden_size) initializes a GRU layer with hidden_size inputs and hidden_size outputs. It is the core layer of the decoder RNN and is responsible for computing the hidden state.

#nn.Linear(hidden_size, output_size) initializes a linear layer with hidden_size inputs and output_size outputs. It is used to convert the hidden state of the RNN to a probability distribution over the output tokens.

#nn.LogSoftmax(dim=1) initializes a log-softmax layer that takes the output of the linear layer and normalizes it to a probability distribution.

#The forward method takes two inputs, an input token and the previous hidden state of the RNN, and returns the output token and the updated hidden state.

#The input token is first passed through the embedding layer, then passed through a ReLU activation function. The resulting output is then passed through the GRU layer along with the previous hidden state. The resulting output from the GRU layer is passed through the linear layer and then through the log-softmax layer to get the probability distribution over the output tokens. The output token with the highest probability is returned as the predicted token for the current step.

#The initHidden method initializes the hidden state to a tensor of zeros with the shape (1, 1, hidden_size) and returns it.

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
#This code defines a decoder RNN with attention mechanism. The attention mechanism is used to help the decoder focus on different parts of the input sequence during decoding. This is particularly useful when dealing with long input sequences.

#The AttnDecoderRNN class takes as input the hidden_size, output_size, dropout_p, and max_length hyperparameters. The hidden_size is the size of the hidden state of the decoder, output_size is the size of the output vocabulary, dropout_p is the probability of dropout, and max_length is the maximum length of the input sequence.

#In the constructor, the class initializes several layers that are used in the forward method. These layers include an embedding layer, an attention layer, an attention-combination layer, a dropout layer, a GRU layer, and an output layer.

#The forward method takes as input the current token input, the previous hidden state hidden, and the encoder outputs encoder_outputs.

#First, the input token is passed through the embedding layer and then through a dropout layer. The dropout layer randomly zeroes some of the elements of the input tensor to prevent overfitting.

#Next, the attention mechanism is applied to the embedded input and the previous hidden state. The attention layer computes a weight for each encoder output based on how well it matches the input and hidden state. The attention-combination layer then combines the attention weights with the encoder outputs to produce a context vector.

#The context vector and the embedded input are concatenated together and passed through the GRU layer to compute the new hidden state. Finally, the output of the GRU layer is passed through a linear layer and a log-softmax layer to produce the output probabilities.

#The initHidden method initializes the hidden state of the decoder RNN with zeros.






In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

5m 0s (- 70m 10s) (5000 6%) 2.8300
9m 57s (- 64m 43s) (10000 13%) 2.2585
15m 1s (- 60m 4s) (15000 20%) 1.9680
20m 1s (- 55m 4s) (20000 26%) 1.7034
25m 1s (- 50m 3s) (25000 33%) 1.5601
29m 59s (- 44m 59s) (30000 40%) 1.3777
35m 0s (- 40m 0s) (35000 46%) 1.2465
40m 1s (- 35m 0s) (40000 53%) 1.1151
56m 26s (- 37m 37s) (45000 60%) 1.0188
61m 28s (- 30m 44s) (50000 66%) 0.9150
66m 33s (- 24m 12s) (55000 73%) 0.8246
71m 34s (- 17m 53s) (60000 80%) 0.7638
76m 41s (- 11m 47s) (65000 86%) 0.7070
81m 41s (- 5m 50s) (70000 93%) 0.6395
86m 41s (- 0m 0s) (75000 100%) 0.5885


In [18]:
evaluateRandomly(encoder1, attn_decoder1)

> nous sommes a court de biere .
= we re out of beer .
< we re out of . . <EOS>

> vous etes incroyablement talentueux .
= you re incredibly talented .
< you re incredibly talented . <EOS>

> nous ne sommes pas en securite .
= we re not safe .
< we re not safe safe . <EOS>

> je suis desolee si je vous ai embarrassee .
= i m sorry if i embarrassed you .
< i m sorry if i embarrassed you . <EOS>

> tu es une opportuniste .
= you re opportunistic .
< you re opportunistic . <EOS>

> je suis libre aujourd hui .
= i m free today .
< i m free today . <EOS>

> je perds patience avec toi .
= i am losing my patience with you .
< i am losing my patience with you . <EOS>

> je lis une lettre .
= i am reading a letter .
< i am reading a letter . <EOS>

> tu en fais partie .
= you re part of this .
< you re part of them . <EOS>

> vous etes l elue .
= you are the chosen one .
< you are the chosen one . <EOS>



In [19]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())


In [73]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent")
evaluateAndShowAttention("je suis ravi de vous rencontrer")
evaluateAndShowAttention("sois un homme")

input = elle a cinq ans de moins que moi .
output = she is five years younger than me . <EOS>
input = elle est trop petit .
output = she s too short . <EOS>
input = je ne crains pas de mourir .
output = i m not scared to die . <EOS>
input = c est un jeune directeur plein de talent
output = he s a talented young director . <EOS>


C:\Users\malhotralovish\AppData\Local\Temp\ipykernel_956\522407878.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\malhotralovish\AppData\Local\Temp\ipykernel_956\522407878.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
C:\Users\malhotralovish\AppData\Local\Temp\ipykernel_956\522407878.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


input = je suis ravi de vous rencontrer
output = i m delighted to meet you . <EOS>
input = sois un homme
output = i m a . <EOS>
